In [9]:
import os
import shutil
from pathlib import Path

def flatten_jpgs(source_dir, destination_dir):
    """
    Recursively find all .jpg files in source_dir and copy them into destination_dir, flattening the structure.
    """
    source = Path(source_dir)
    destination = Path(destination_dir)

    if not source.is_dir():
        raise NotADirectoryError(f"Source directory does not exist: {source}")
    destination.mkdir(parents=True, exist_ok=True)

    jpg_files = list(source.rglob("*.jpg")) + list(source.rglob("*.JPG"))
    copied = 0
    errors = 0

    for file in jpg_files:
        try:
            # If duplicate filename exists, add a numeric suffix
            target_file = destination / file.name
            counter = 1
            while target_file.exists():
                stem = file.stem
                suffix = file.suffix
                target_file = destination / f"{stem}_{counter}{suffix}"
                counter += 1

            shutil.copy2(file, target_file)
            copied += 1

        except Exception as e:
            print(f"Error copying {file}: {e}")
            errors += 1

    print(f"Copied {copied} files to '{destination}'.")
    if errors:
        print(f"{errors} file(s) failed to copy.")

# Example usage in a Jupyter Notebook:
# flatten_jpgs("/Users/erikborn/Documents/Python/LifeTouch Photos", "/Users/erikborn/Documents/Python/Misc/LifeTouchPost")

In [10]:
flatten_jpgs("/Users/erikborn/Documents/Python/Pixevity/LifeTouch Photos", "/Users/erikborn/Documents/Python/Pixevity/LifeTouchPost")

Copied 802 files to '/Users/erikborn/Documents/Python/Pixevity/LifeTouchPost'.


In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path

def rename_or_move_images(image_folder, csv_mapping_path, unmatched_folder, default_extension=".jpg", case_insensitive=True):
    image_folder = Path(image_folder)
    unmatched_folder = Path(unmatched_folder)
    unmatched_folder.mkdir(parents=True, exist_ok=True)

    # Load CSV with mapping
    try:
        df = pd.read_csv(csv_mapping_path)
    except Exception as e:
        raise ValueError(f"Failed to load CSV: {e}")

    if df.shape[1] < 2:
        raise ValueError("CSV must have at least two columns: old filename, new filename")

    old_col, new_col = df.columns[0], df.columns[1]

    # Normalize mapping keys for case-insensitive matching if requested
    if case_insensitive:
        mapping = {str(k).strip().lower(): str(v).strip() for k, v in zip(df[old_col], df[new_col])}
    else:
        mapping = {str(k).strip(): str(v).strip() for k, v in zip(df[old_col], df[new_col])}

    renamed_count = 0
    moved_count = 0
    seen_files = 0
    matched_files = 0

    for file in image_folder.iterdir():
        if not file.is_file():
            continue

        seen_files += 1
        old_name = file.name.strip()
        old_name_key = old_name.lower() if case_insensitive else old_name

        if old_name_key in mapping:
            matched_files += 1
            new_name = mapping[old_name_key]

            # Ensure extension
            if not Path(new_name).suffix:
                new_name += default_extension

            new_path = image_folder / new_name
            counter = 1
            while new_path.exists():
                stem = Path(new_name).stem
                suffix = Path(new_name).suffix
                new_path = image_folder / f"{stem}_{counter}{suffix}"
                counter += 1

            try:
                file.rename(new_path)
                renamed_count += 1
            except Exception as e:
                print(f"Failed to rename {old_name} → {new_name}: {e}")

        else:
            try:
                shutil.move(str(file), unmatched_folder / file.name)
                moved_count += 1
            except Exception as e:
                print(f"Failed to move unmatched file {old_name}: {e}")

    print("=== Summary ===")
    print(f"Files scanned      : {seen_files}")
    print(f"Matches found      : {matched_files}")
    print(f"Files renamed      : {renamed_count}")
    print(f"Files unmatched    : {moved_count}")
    print(f"Expected mappings  : {len(mapping)}")

In [11]:
rename_or_move_images(
    image_folder="/Users/erikborn/Documents/Python/Pixevity/LifeTouchPost",
    csv_mapping_path="/Users/erikborn/Documents/Python/Pixevity/map.csv",
    unmatched_folder="/Users/erikborn/Documents/Python/Pixevity/NoUse"
)

=== Summary ===
Files scanned      : 803
Matches found      : 741
Files renamed      : 741
Files unmatched    : 62
Expected mappings  : 741
